# Crawl Data from Top Youtuber in Taiwan

##  1. Get Famous Youtuber Link

In [1]:
'''
###########################################################
    Import Libraries
###########################################################
'''
import numpy as np
import requests
import sys
import math
import time
from bs4 import BeautifulSoup
import functools
import csv
import pandas as pd

### Get the top 250 youtubers link and basic attributes in taiwan. Sorted by subscribers num

In [2]:
'''
######################################################################################################
    Get the top 250 youtubers link in socialblade website
######################################################################################################
'''

top_youtuber_rank_url = "https://socialblade.com/youtube/top/country/tw/mostsubscribed"

content = requests.get(top_youtuber_rank_url).text
page_soup = BeautifulSoup(content, "html.parser")

# Find potential user position 
div_class = page_soup.find_all('a', href=True)

youtuber_socialblade = [] # Name, url in socialblade
for num in range(0, len(div_class)):
    href = div_class[num]
    url = href.get('href')
    if url.find("/youtube/user/") is not -1:
        youtuber_socialblade.append([href.text, "https://socialblade.com"+url])

print("Top 250 Youtuber: ")
for i in range(0, len(youtuber_socialblade)):
    print(i+1, 'is', youtuber_socialblade[i][0], youtuber_socialblade[i][1])

Top 250 Youtuber: 
1 is 這群人TGOP https://socialblade.com/youtube/user/e84768131
2 is 滾石唱片 ROCK RECORDS https://socialblade.com/youtube/user/rockrecordstaipei
3 is 相信音樂BinMusic https://socialblade.com/youtube/user/binmusictaipei
4 is 阿滴英文 https://socialblade.com/youtube/user/eo3jwe3hezuwfdvcehqk9q
5 is 蔡阿嘎 https://socialblade.com/youtube/user/kyoko38
6 is 阿神 https://socialblade.com/youtube/user/charlie615119
7 is 杰威爾音樂 JVR Music https://socialblade.com/youtube/user/jvrmuzic
8 is 華納音樂 Warner Music Taiwan https://socialblade.com/youtube/user/warnertaiwan
9 is 聖結石Saint https://socialblade.com/youtube/user/idhd_1spj49unbwx1fjs2a
10 is 眾量級CROWD https://socialblade.com/youtube/user/3lbfxbwtebdooub8-qjm9q
11 is 瑪莎與熊 Masha and The Bear CH https://socialblade.com/youtube/user/fsxflbi8fpnavq_rl3i4bw
12 is 福茂唱片 https://socialblade.com/youtube/user/linfairrecords
13 is 華研國際 https://socialblade.com/youtube/user/himservice
14 is 放火 Louis https://socialblade.com/youtube/user/louislee0602
15 is 小玉 https

In [3]:
'''
######################################################################################################
    Get youtuber name, url, UPLOADS, SUBSCRIBERS, VIDEO VIEWS, Country, CHANNEL TYPE, USER CREATED DATE
######################################################################################################
'''
youtuber = []
for i in range(0, len(youtuber_socialblade)): 
    user = youtuber_socialblade[i]
    url = user[1]
    content = requests.get(url).text
    page_soup = BeautifulSoup(content, "html.parser")
    attributes = []
    attributes.append(user[0])
    
    # Get youtube url
    youtube_link_class = page_soup.find_all("a", {"class": "core-button -margin core-small-wide ui-black"})
    for link_text in youtube_link_class:
        link_url = link_text.get('href')
        if link_url is not None:
            attributes.append(link_url)
    
    # Get attributes value
    info_class = page_soup.find_all("div", {"class": "YouTubeUserTopInfo"})
    for info in info_class:
        value = info.find_all("span", {"style": "font-weight: bold;"})[0].text
        attributes.append(value)
    youtuber.append(attributes)
    time.sleep(0.05)
print("Done")

Done


In [4]:
print(type(youtuber))
for i in range(0, len(youtuber)):
    for j in youtuber[i]:
        print(j, end=" "),
    print()

<class 'list'>
這群人TGOP https://youtube.com/channel/UC6FcYHEm7SO1jpu5TKjNXEA 177 2,796,998 598,513,144 TW Entertainment Jun 7th, 2008 
滾石唱片 ROCK RECORDS https://youtube.com/channel/UCPcF3KTqhD67ADkukx_OeDg 4,278 2,233,857 2,552,567,391 TW Music Jun 8th, 2010 
相信音樂BinMusic https://youtube.com/channel/UC7ovu6a8ydIbDy0fAKmoZ9A 1,209 2,122,546 2,304,859,244 TW Music Aug 10th, 2009 
阿滴英文 https://youtube.com/channel/UCeo3JwE3HezUWFdVcehQk9Q 353 1,965,923 144,397,071 TW Education Jan 11th, 2015 
蔡阿嘎 https://youtube.com/channel/UCPwxSX0DYDMQxCvgfeVDv_g 549 1,936,842 422,794,657 TW Entertainment Jun 3rd, 2006 
阿神 https://youtube.com/channel/UCnJEWsS5agXCkqIpyHC9Grg 3,524 1,732,081 656,947,095 TW Games May 11th, 2009 
杰威爾音樂 JVR Music https://youtube.com/channel/UCKUlsqazP-4QmxdEtUPlxOA 516 1,451,103 2,031,504,956 TW Music Sep 2nd, 2010 
華納音樂 Warner Music Taiwan https://youtube.com/channel/UCStpPtoeQUYII1Jsbl0qf8A 1,700 1,419,391 907,971,142 TW Music Jul 6th, 2009 
聖結石Saint https://youtube.com/cha

In [5]:
'''
######################################################################################################
    Preprocess data about comma and date
######################################################################################################
'''
month_dictionary = {'Jan':'01', 'Feb':'02', "Mar":'03', "Apr":'04', "May":'05', "Jun":'06', 
                    "Jul":'07', "Aug":'08', "Sep":'09', "Oct":'10', "Nov":'11', "Dec":'12'}
date_dictionary = {'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07', '8':'08', '9':'09'}

def remove_comma(value_string):
    if type(value_string) == int:
        return value_string
    value = 0
    for i in range(0, len(value_string)):
        if(value_string[i]!=","):
            value = value*10 + int(value_string[i])
    return int(value)
       
def date_to_normal_express(value_string):
    if type(value_string) == int or len(value_string) == 10:
        return value_string
    value_string = value_string.replace(',', " ")
    value_string = value_string.replace('th', "")
    value_string = value_string.replace('rd', "")
    value_string = value_string.replace('nd', "")
    value_string = value_string.replace('st', "")
    
    date = value_string.split()
    date[0] = month_dictionary[date[0]]
    if(len(date[1]) == 1):
        date[1] = date_dictionary[date[1]]
    
    date = date[2] + "-" + date[0] + "-" + date[1]
    
    return (date)

today = "2018-12-26"

for i in range(0, len(youtuber)):
    youtuber[i][0] = youtuber[i][0].replace(';', "")
    youtuber[i][2] = remove_comma(youtuber[i][2]) #UPLOADS
    youtuber[i][3] = remove_comma(youtuber[i][3]) #SUBSCRIBERS
    youtuber[i][4] = remove_comma(youtuber[i][4]) #VIDEO VIEWS
    youtuber[i][7] = date_to_normal_express(youtuber[i][7]) #Date from string to int
    youtuber[i].append(today)
    
    # calculate the channel founded dates
    start_date = youtuber[i][7]
    end_date = today
    start_sec = time.mktime(time.strptime(start_date,'%Y-%m-%d'))
    end_sec = time.mktime(time.strptime(end_date,'%Y-%m-%d'))
    work_days = int((end_sec - start_sec)/(24*60*60))
    youtuber[i].append(work_days)

    
for i in range(0, len(youtuber)):
    for j in youtuber[i]:
        print(j, end=" "),
    print()

這群人TGOP https://youtube.com/channel/UC6FcYHEm7SO1jpu5TKjNXEA 177 2796998 598513144 TW Entertainment 2008-06-07 2018-12-26 3854 
滾石唱片 ROCK RECORDS https://youtube.com/channel/UCPcF3KTqhD67ADkukx_OeDg 4278 2233857 2552567391 TW Music 2010-06-08 2018-12-26 3123 
相信音樂BinMusic https://youtube.com/channel/UC7ovu6a8ydIbDy0fAKmoZ9A 1209 2122546 2304859244 TW Music 2009-08-10 2018-12-26 3425 
阿滴英文 https://youtube.com/channel/UCeo3JwE3HezUWFdVcehQk9Q 353 1965923 144397071 TW Education 2015-01-11 2018-12-26 1445 
蔡阿嘎 https://youtube.com/channel/UCPwxSX0DYDMQxCvgfeVDv_g 549 1936842 422794657 TW Entertainment 2006-06-03 2018-12-26 4589 
阿神 https://youtube.com/channel/UCnJEWsS5agXCkqIpyHC9Grg 3524 1732081 656947095 TW Games 2009-05-11 2018-12-26 3516 
杰威爾音樂 JVR Music https://youtube.com/channel/UCKUlsqazP-4QmxdEtUPlxOA 516 1451103 2031504956 TW Music 2010-09-02 2018-12-26 3037 
華納音樂 Warner Music Taiwan https://youtube.com/channel/UCStpPtoeQUYII1Jsbl0qf8A 1700 1419391 907971142 TW Music 2009-07-06 20

超粒方 https://youtube.com/channel/UC0Q-fBheHysYWz9ObSEzMdA 101 543686 24685529 TW Film 2014-10-17 2018-12-26 1531 
SETN三立新聞網 https://youtube.com/channel/UCIU8ha-NHmLjtUwU7dFiXUA 48768 539007 979278522 TW People 2014-01-05 2018-12-26 1816 
冏星人 https://youtube.com/channel/UC1tp9qN7mqIJ7CLVNVvsKzg 141 536831 29327389 TW Film 2013-04-18 2018-12-26 2078 
台灣蘋果日報 https://youtube.com/channel/UChLEhd2gouo1H-c9Zy9WGDA 55523 524781 978610297 TW News 2010-12-16 2018-12-26 2932 
黑男 https://youtube.com/channel/UCnMlG9P337rFzaJVly5EZDw 1151 523012 187614488 TW Entertainment 2014-05-13 2018-12-26 1688 
蔡佩軒 Ariel Tsai https://youtube.com/channel/UCt1g7JuMPxb1NYRvn3JCA4Q 76 522041 88759019 TW Music 2008-08-04 2018-12-26 3796 
楊桃美食網 https://youtube.com/channel/UCctVKh07hVAyQtqpl75pxYA 2373 521692 174951626 TW Howto 2013-07-30 2018-12-26 1975 
YOYOTV https://youtube.com/channel/UCiWRSesvSYmY7YOyz0tv_zQ 7593 521099 606022345 TW Education 2009-12-04 2018-12-26 3309 
七不哒哒 https://youtube.com/channel/UCIQhxw3Lv

In [6]:
print(len(youtuber[0][7]))

10


## 2. Write basic information of youtuber

In [7]:
'''
######################################################################################################
    Write the basic information of youtuber to csv file
######################################################################################################
'''

with open('youtuber.csv', 'w+', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['User_Name', 'Link', 'Uploads', 'Subscribers', 'Total_Video_Views', 'Country', \
                        'Channel_Type', 'User_Created_Date', "Today_Date", 'User_Exist_Days'])
    for i in range(len(youtuber)):
        decode = []
        for info in youtuber[i]:
            if type(info) == str:
                decode.append(info.encode(sys.stdin.encoding, "replace").decode(sys.stdin.encoding))
            else:
                decode.append(info)
        #print(decode)
        writer.writerow(decode)
print("Done")

Done
